# Acta No. 17 – Reunión de asesoría


**Proyecto:** Consolidación del modelo SARIMAX para la predicción de epidemias de dengue en el municipio de Caucasia



**Estudiante:** Janis Zúñiga Ortega


**Asesor:** Marco Julio Cañas Campillo


**Fecha:** Martes 3 de febrero de 2026


**Lugar:** Virtual por meet 



# 1. Contexto y antecedentes



Esta asesoría da continuidad a los acuerdos establecidos en el **Acta No. 16 (26 de enero de 2026)**, en la cual se avanzó en:

* Teneamos el dataset con todos los atributos meteorológicos con los "retrazos" hasta de 12 semanas epidemiológicas y los dejaremos a 10. 


* Realizó las visualizaciones de las correlaciones en cada semana de cada atributu meteorológico. 
  


## Terea 

Lo que hay por hacer: 

* El análisis de correlación (Pearson, Kendall y Spearman) entre dichas variables y el número de casos de dengue (`num_casos`).
* Hay que establecer, ya formalmente, los parametros p, d, q y P, D, Q, s



# Referente para establecer lo que sigue en este trabajo  



Sí, **es no solo bueno sino prácticamente obligatorio** hacer **preprocesamiento de datos** antes de consolidar un modelo **SARIMAX**.
Un SARIMAX *sin* buen preprocesamiento suele dar **parámetros inestables, residuos autocorrelados y mala capacidad predictiva**, aunque el código “corra”.

Te lo organizo **conceptual → práctico → recomendado para tu contexto (dengue + clima)**.

---

## 1️⃣ ¿Por qué el SARIMAX necesita preprocesamiento?

Un SARIMAX asume (explícita o implícitamente):

* **Estacionariedad** (en media y varianza)
* **Estructura temporal limpia**
* **Relaciones lineales** entre la serie objetivo y los exógenos
* **Rezagos bien definidos**

Los datos reales (epidemiológicos y meteorológicos) **violan todo eso** por defecto 😅.

---

## 2️⃣ Preprocesamiento esencial (sí o sí)

### 🔹 1. Limpieza y alineación temporal

Antes de cualquier modelo:

```python
df = df.sort_index()
df = df.asfreq('W')   # o 'D', 'M' según tu caso
```

✔️ Índice temporal regular
✔️ Sin fechas duplicadas

---

### 🔹 2. Tratamiento de datos faltantes

SARIMAX **no tolera NaN**.

Recomendado:

```python
df = df.interpolate(method='time')
```

O, si es semanal:

```python
df = df.fillna(method='ffill')
```

📌 Nunca entrenar SARIMAX con NaN.

---

### 🔹 3. Transformaciones de escala (varianza)

Muy común en dengue:

```python
import numpy as np

df['casos_log'] = np.log1p(df['casos_dengue'])
```

✔️ Reduce heterocedasticidad
✔️ Mejora estabilidad numérica

---

## 3️⃣ Estacionariedad (el corazón del SARIMAX)

### 🔹 4. Diferenciación regular (d)

```python
df['casos_diff'] = df['casos_log'].diff()
```

### 🔹 5. Diferenciación estacional (D)

Si hay estacionalidad anual (datos semanales):

```python
df['casos_diff_seasonal'] = df['casos_diff'].diff(52)
```

📌 En SARIMAX, esto se declara como:

```python
order=(p, d, q)
seasonal_order=(P, D, Q, s)
```

---

## 4️⃣ Preprocesamiento de variables exógenas (X)

Esto es **crítico** y a menudo se hace mal.

### 🔹 6. Selección por correlación cruzada (ccf)

```python
from statsmodels.tsa.stattools import ccf

ccf_vals = ccf(df['lluvia'].diff().dropna(),
               df['casos_dengue'].diff().dropna())
```

✔️ Identifica rezagos informativos

---

### 🔹 7. Crear rezagos explícitos

```python
for lag in [1, 2, 4, 8]:
    df[f'lluvia_lag_{lag}'] = df['lluvia'].shift(lag)
```

📌 SARIMAX **NO descubre rezagos automáticamente**.

---

### 🔹 8. Escalado de exógenos (recomendado)

```python
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[exog_cols] = scaler.fit_transform(df[exog_cols])
```

✔️ Evita dominancia numérica
✔️ Mejora convergencia

---

## 5️⃣ Control de colinealidad

Variables climáticas suelen estar correlacionadas:

```python
df[exog_cols].corr()
```

📌 Alta colinealidad ⇒ coeficientes inestables.

✔️ Elimina variables redundantes.

---

## 6️⃣ Diagnóstico previo al modelado

Antes de ajustar SARIMAX:

* ACF / PACF
* Prueba ADF
* Inspección visual

```python
from statsmodels.tsa.stattools import adfuller
adfuller(df['casos_diff'].dropna())
```

---



# 7️⃣ Pipeline recomendado (resumen)

Para **consolidar un SARIMAX sólido**:

1. Limpieza y frecuencia temporal
2. Imputación de faltantes
3. Transformación log / Box-Cox
4. Diferenciación regular y estacional
5. Selección de exógenos con `ccf`
6. Construcción explícita de rezagos
7. Escalado de exógenos
8. Diagnóstico de residuos

---



## 8️⃣ En una frase (para artículo)

> *“Se realizó un preprocesamiento exhaustivo que incluyó imputación temporal, transformación de escala, diferenciación regular y estacional, selección de rezagos mediante correlación cruzada y estandarización de variables exógenas, con el fin de garantizar estacionariedad, estabilidad numérica y capacidad predictiva del modelo SARIMAX.”*

---

Si quieres, en el siguiente mensaje puedo:

* construir **un pipeline SARIMAX completo en Python**,
* ayudarte a **justificar metodológicamente** cada paso para un artículo,
* o comparar **SARIMAX vs ML** usando exactamente el mismo preprocesamiento.
